___
# Ciência dos Dados - Projeto 1 #
___

### _Gabriela Caruso e Mayra Peter - 2C_ ###
___

## Introdução: _Classificador Automático de Sentimento_ ##

_Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing._

_Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo._

_Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma._

___

## Objetivo ##

Visando analisar o conteúdo de _tweets_ e retornar se eles são ou não relevantes, foi criado um programa que percorre uma planilha de dados coletando informações e, por fim, retorna a  relevância dos dados a partir de suas respectivas probabilidades de ocorrência.
___

In [1]:
%matplotlib inline
%reset -f## Coleta e seleção de dados ##
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from emoji import UNICODE_EMOJI

## Coleta e seleção de dados ##

Foi criado um _dataframe_ contendo 500 _tweets_. Posteriormente foi classificado manualmente a relevância de cada um, sendo 0 (zero) não relevante e 1 (um) relevante.

Na célula abaixo está sendo importado o _dataframe_ inicial e separado em "Treinamento" (300 _tweets_), a partir do qual serão coletadas as probabilidades, e em "Teste" (200 _tweets_), no qual o programa analisará os dados coletados em "Treinamento" para calcular a relevância desses 200 _tweets_.
___

In [3]:
data = pd.ExcelFile('tweets_Tinder_201809041724.xlsx')
data_treinamento = pd.read_excel(data, 'Treinamento')
data_teste = pd.read_excel(data, 'Teste')

## Limpeza dos dados ##

Para que o programa não analise partes desnecessárias dos _tweets_, os dados foram filtrados, retirando pontuações e simbolos não relevantes e arrumando espaçamentos incorretos (incluindo entre _emojis_).

___

In [7]:
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace(',', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('.', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace(':', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('/', ' ')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('#', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('@', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('?', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('\n', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('rt', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('(', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace(')', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('*', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('“', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('”', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('_', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('&', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace(';', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('%', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace("'", '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('-', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('+', '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace("=", '')
data_treinamento.Treinamento = data_treinamento.Treinamento.str.replace('"', '')

data_teste.Teste = data_teste.Teste.str.replace(',', '')
data_teste.Teste = data_teste.Teste.str.replace('.', '')
data_teste.Teste = data_teste.Teste.str.replace(':', '')
data_teste.Teste = data_teste.Teste.str.replace('/', ' ')
data_teste.Teste = data_teste.Teste.str.replace('#', '')
data_teste.Teste = data_teste.Teste.str.replace('@', '')
data_teste.Teste = data_teste.Teste.str.replace('?', '')
data_teste.Teste = data_teste.Teste.str.replace('rt', '')
data_teste.Teste = data_teste.Teste.str.replace('\n', '')
data_teste.Teste = data_teste.Teste.str.replace('(', '')
data_teste.Teste = data_teste.Teste.str.replace(')', '')
data_teste.Teste = data_teste.Teste.str.replace('*', '')
data_teste.Teste = data_teste.Teste.str.replace('“', ' ')
data_teste.Teste = data_teste.Teste.str.replace('”', '')
data_teste.Teste = data_teste.Teste.str.replace('_', '')
data_teste.Teste = data_teste.Teste.str.replace('&', '')
data_teste.Teste = data_teste.Teste.str.replace(';', '')
data_teste.Teste = data_teste.Teste.str.replace('%', '')
data_teste.Teste = data_teste.Teste.str.replace("'", '')
data_teste.Teste = data_teste.Teste.str.replace('-', '')
data_teste.Teste = data_teste.Teste.str.replace('+', '')
data_teste.Teste = data_teste.Teste.str.replace('=', ' ')
data_teste.Teste = data_teste.Teste.str.replace('"', '')

lista_espaco = []
for each in data_treinamento['Treinamento']:
    e = ''
    for word in each:
        if word in UNICODE_EMOJI:
            e += " " + word + ' '
        else:
            e += word
    lista_espaco.append(e)
    
data_treinamento['Treinamento'] = lista_espaco

lista_espaco2 = []
for each in data_teste['Teste']:
    e = ''
    for word in each:
        if word in UNICODE_EMOJI:
            e += " " + word + ' '
        else:
            e += word
    lista_espaco2.append(e)
    
data_teste['Teste'] = lista_espaco2

## Separação dos dados ##

Para facilitar o cálculo das probabilidades, a partir da planilha de treinamento, foram criadas duas novas planilhas, uma contendo os _tweets_ relevantes e outra os irrelevantes.

___

In [11]:
data_treinamento_relevantes = data_treinamento[(data_treinamento['Relevância'] == 1)]
data_treinamento_irrelevantes = data_treinamento[(data_treinamento['Relevância'] == 0)]

In [10]:
data_treinamento_relevantes.head()

,Treinamento,Relevância
2,terceira vez que dou match com a mesma pessoa ...,1
5,eduaaarda eu n sei pq ainda insisto em instal...,1
6,tal qual o ubero tinder tinha que ter uma área...,1
12,karlasnotsorry gente eu criei um tinder da no...,1
13,karlasnotsorry gente eu criei um tinder da no...,1


In [12]:
data_treinamento_irrelevantes.head()

,Treinamento,Relevância
0,o cara do tinder ficou brabo cmg pq eu disse q...,0
1,fica a questão tinder https tco iq7iyzucdd,0
3,meta pra 2018 deletar o tinder ainda esse ano ...,0
4,para todos os matches do tinder que eu não man...,0
7,to muito confuso hoje to tipo gente que namora...,0


## Criando listas ##

Para separar as palavras, foram criadas três listas. A primeira contendo todas as palavras, a uma segunda contendo as palavras relevantes e outra com as irrelevantes.

___